### References
- https://jovian.ai/aakashns/01-pytorch-basics
- https://jovian.ai/aakashns/02-linear-regression

### Setup

In [1]:
import torch
import numpy as np
from IPython.display import display
import pandas as pd

### gradとは
- 微分対象にしたい変数に、requires_grad=Trueをする。
- これらを使用した演算結果は、backwardできるように勝手になる。
- backwardは誤差逆伝搬のこと。
- 機械学習のパラメータは、再急降下法により更新され、最適値を目指す。
- loss関数を各パラメータで偏微分した値でパラメータを更新する。
  - lossをy、parametersをX、更新後のparametersをX'とすると、更新式は以下となる。
  - X'[t] = X[t] - a * dy / dX[t]
  - ここで、aはいわゆるlearning_rateである。

In [2]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True) # これで微分対象となる
b = torch.tensor(5., requires_grad=True) # これで微分対象となる
x, w, b

y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

- くらえ！自動微分！！

In [3]:
y.backward()
display(f"dy/dx = {x.grad}, dy/dw = {w.grad}, dy/db = {b.grad}")

'dy/dx = None, dy/dw = 3.0, dy/db = 1.0'

- ちなみに微分前に、微分を覗くとNoneになっている。

In [4]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True) # これで微分対象となる
b = torch.tensor(5., requires_grad=True) # これで微分対象となる
print(f"dy/dx = {x.grad}, dy/dw = {w.grad}, dy/db = {b.grad}")

dy/dx = None, dy/dw = None, dy/db = None


### 線形回帰
- 何かyを推定する際に、入力情報の線形結合でモデル化する場合、これを線形モデルという。
  - モデルとしてはこんな感じ。なんかの係数と入力ベクトルの和で表現されるなら全部線形モデル。
    - y' = b + a0 * x0 + a1 * x1 ...
  - よく、線形は１次関数とか言われますけど、２次関数も線形モデルです。
  - なぜなら、以下のような２次関数も入力の線形結合だから。
    - y' = b + a0 * x + a1 * x * x
- せっかくなんで、pandas使ってみる。

In [5]:
df = pd.DataFrame([[73, 67, 43, 56, 70], 
                   [91, 88, 64, 81, 101], 
                   [87, 134, 58, 119, 133], 
                   [102, 43, 37, 22, 37], 
                   [69, 96, 70, 103, 119]], columns=["温度", "降水量", "湿度", "apples", "oranges"])
display(df[["温度", "降水量", "湿度"]])

,温度,降水量,湿度
0,73,67,43
1,91,88,64
2,87,134,58
3,102,43,37
4,69,96,70


- 入力と正解をtensorにする。
  - pandas -> numpy -> tensorに変換する。

In [6]:
inputs = torch.from_numpy(df[["温度", "降水量", "湿度"]].values.astype(np.float32))
display(inputs.shape)

targets = torch.from_numpy(df[["apples", "oranges"]].values.astype(np.float32))
display(targets.shape)

torch.Size([5, 3])

torch.Size([5, 2])

- パラメータ初期化

In [7]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
display(w,b)

tensor([[ 0.9026, -2.3861,  0.9195],
        [ 0.5287,  0.8454, -1.3245]], requires_grad=True)

tensor([ 1.8754, -0.3170], requires_grad=True)

- モデル定義
  - 初期値はランダムなので、この時点で推論しても意味不明である。
  - .t()は転置になるらしい。

In [8]:
def model(x):
    return x @ w.t() + b

preds = model(inputs)
display(preds, targets)

tensor([[ -52.5683,   37.9669],
        [ -67.1211,   37.4227],
        [-186.0094,   82.1422],
        [  25.3562,   40.9574],
        [-100.5497,   24.6072]], grad_fn=<AddBackward0>)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

- 誤差関数定義
  - 学習の目標とする、最小化したい指標を定義する。
  - .numel()は要素数になるらしい。

In [9]:
# MSE ... Mean squared error
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

loss = mse(preds, targets)
display(loss)

tensor(18478.1816, grad_fn=<DivBackward0>)

- 再度、微分じゃ！くらえ！

In [10]:
loss.backward()

In [11]:
display(w) # これは前と一緒
display(w.grad)

tensor([[ 0.9026, -2.3861,  0.9195],
        [ 0.5287,  0.8454, -1.3245]], requires_grad=True)

tensor([[-12328.5840, -16115.2910,  -9192.6074],
        [ -3731.6057,  -4689.5024,  -2971.4397]])

- 微分を元に再急降下法の式で更新する。
- この更新計算時は、微分は不要なので、no_gradというcontextmanagerが必要。
- contextmanagerは自分で作成もできる。以下を参考。
  - https://qiita.com/QUANON/items/c5868b6c65f8062f5876

In [12]:
# 1e-5がlearning_rateである。
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

- 更新が終わったら、次の更新に備えて、gradをゼロに戻しておいた方が良い。

In [13]:
w.grad.zero_()
b.grad.zero_()
display(w.grad, b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

tensor([0., 0.])

- 更新後の値で、再度推論をし、誤差を計算しなおして、誤差が減ることを確認します。

In [14]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(13549.3389, grad_fn=<DivBackward0>)


### 複数回のepochで学習
- 複数回実行する形式で振り返ります。
- トータルとしてこういう流れが必要です。
- 少し見た目もおしゃれにしました。

In [15]:
import time
from tqdm.notebook import tqdm

# 入力
inputs = torch.from_numpy(df[["温度", "降水量", "湿度"]].values.astype(np.float32))

# 出力(正解)
targets = torch.from_numpy(df[["apples", "oranges"]].values.astype(np.float32))

# パラメータ初期化
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

# モデル定義
def model(x):
    return x @ w.t() + b

# 誤差関数定義
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

# Train for 100 epochs
with tqdm(range(100)) as pbar:
    for i in pbar:
        time.sleep(0.05)
        preds = model(inputs)
        loss = mse(preds, targets)
        loss.backward()
        with torch.no_grad():
            w -= w.grad * 1e-5
            b -= b.grad * 1e-5
            w.grad.zero_()
            b.grad.zero_()
        
        pbar.set_description(f"[loss: {loss:.1f}]")

  0%|          | 0/100 [00:00<?, ?it/s]

- predsとtargetsの値を比較してみます。

In [16]:
preds = model(inputs)
display(preds, targets)

tensor([[ 58.4133,  73.1008],
        [ 79.2721, 101.9216],
        [123.3582, 125.6232],
        [ 28.8668,  53.4883],
        [ 92.0867, 111.6388]], grad_fn=<AddBackward0>)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

### PyTorchのビルトインの場合
- ここまでやってきたことは、標準のクラスで実現できる。
- データ読み込みは、ミニバッチ分割・シャッフルなどをDataLoaderに任せられる。
  - ここまでバッチ分割せず、全データでパラメータを更新していた。
  - バッチ分割とは、データをある単位で分割してそれぞれでパラメータを更新すること。
  - これを普通の再急降下法ではなく、確率的勾配降下法(SGD: Statistical Gradient Descent)と呼ぶ
- ちなみにSGDはDeep Learningのブレイクスルー要因のひとつだった気がする。
- DataLoaderはgeneratorである。

In [17]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# 入力
inputs = torch.from_numpy(df[["温度", "降水量", "湿度"]].values.astype(np.float32))

# 出力(正解)
targets = torch.from_numpy(df[["apples", "oranges"]].values.astype(np.float32))

train_ds = TensorDataset(inputs, targets)
train_dl = train_dl = DataLoader(train_ds, batch_size=2, shuffle=True)
display([*train_dl])

[[tensor([[91., 88., 64.],
          [73., 67., 43.]]),
  tensor([[ 81., 101.],
          [ 56.,  70.]])],
 [tensor([[ 69.,  96.,  70.],
          [ 87., 134.,  58.]]),
  tensor([[103., 119.],
          [119., 133.]])],
 [tensor([[102.,  43.,  37.]]), tensor([[22., 37.]])]]

- モデル定義はnnに準備されている。
  - requires_gradも勝手についている。

In [18]:
import torch.nn as nn
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0084, -0.2439,  0.5430],
        [ 0.3842,  0.1587,  0.1924]], requires_grad=True)
Parameter containing:
tensor([-0.3559,  0.2064], requires_grad=True)


- モデルのすべてのパラメータを見たい場合は、こうする。
  - generatorなのでlist()か\[*\]で展開する。

In [19]:
display([*model.parameters()])

[Parameter containing:
 tensor([[-0.0084, -0.2439,  0.5430],
         [ 0.3842,  0.1587,  0.1924]], requires_grad=True),
 Parameter containing:
 tensor([-0.3559,  0.2064], requires_grad=True)]

- 推論
  - まだ学習してないパラメータで。

In [20]:
preds = model(inputs)
preds

tensor([[ 6.0391, 47.1631],
        [12.1692, 61.4533],
        [-2.2753, 66.0630],
        [ 8.3916, 53.3415],
        [13.6606, 55.4249]], grad_fn=<AddmmBackward>)

- 損失関数も代表的なものは準備されている。

In [21]:
import torch.nn.functional as F

loss_fn = F.mse_loss
loss = loss_fn(preds, targets)
loss

tensor(4098.3047, grad_fn=<MseLossBackward>)

- またパラメタの更新方式をoptimizerと呼んだりする。
- 今回は、learning_rate固定のバッチ分割学習なので、普通のSGDとなる。
  - 実際は、learning_rateをかなり工夫して調整する方式がたくさんある。<br>
    Adamなどを使うのがデファクトスタンダートである気がする。

In [22]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### PyTorchのビルトインのまとめ

In [23]:
inputs = torch.from_numpy(df[["温度", "降水量", "湿度"]].values.astype(np.float32))
targets = torch.from_numpy(df[["apples", "oranges"]].values.astype(np.float32))

train_ds = TensorDataset(inputs, targets)
train_dl = DataLoader(train_ds, batch_size=2, shuffle=True)

model = nn.Linear(3, 2)

loss_fn = F.mse_loss

opt = torch.optim.SGD(model.parameters(), lr=1e-5)

def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    with tqdm(range(num_epochs)) as pbar:
    # Repeat for given number of epochs
        for epoch in pbar:

            # バッチ毎の処理
            for xb, yb in train_dl:
                time.sleep(0.01)

                # 推論
                pred = model(xb)

                # 誤差計算
                loss = loss_fn(pred, yb)

                # 微分実行
                loss.backward()

                # モデルのパラメータが更新される
                opt.step()

                # パラメータの自動微分値を0にクリア
                opt.zero_grad()

            pbar.set_description(f"[loss: {loss.item():.1f}]")
            
fit(100, model, loss_fn, opt, train_dl)

  0%|          | 0/100 [00:00<?, ?it/s]

- predsとtargetsの値を比較してみます。

In [24]:
preds = model(inputs)
display(preds, targets)

tensor([[ 58.7726,  71.6741],
        [ 78.5486,  97.9258],
        [124.7171, 136.7875],
        [ 29.3907,  44.4688],
        [ 90.7530, 110.0876]], grad_fn=<AddmmBackward>)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [27]:
import jovian
jovian.commit(project='pytorch-tutorial-02-linear-model', filename='02_linear_model.ipynb')

<IPython.core.display.Javascript object>

[jovian] Updating notebook "nokomoro3/pytorch-tutorial-02-linear-model" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/nokomoro3/pytorch-tutorial-02-linear-model


'https://jovian.ai/nokomoro3/pytorch-tutorial-02-linear-model'